<a href="https://colab.research.google.com/github/jyp1111/TIL/blob/master/jyp/naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [6]:
class naive_bayes:
    def __init__(self):
        self.X_train=None
        self.y_train=None
        self.tokens_ls=None
        self.word_ls=None

    def fit(self,X_train,y_train):
        self.X_train=X_train
        self.y_train=y_train
        self.tokens_ls=[x.split() for x in self.X_train]
        self.word_ls=list(set(sum(self.tokens_ls,[])))

        temp_table=[]
        temp2_table=[]

        for i, tokens in enumerate(self.tokens_ls):
            for word in tokens:
                temp_table.append({"word":word,"doc_cat":i})

        for i in range(len(self.word_ls)):
            count_ls=np.zeros(len(set(self.y_train)))
            for v in temp_table:
                if v["word"]==self.word_ls[i]:
                    count_ls[self.y_train[v["doc_cat"]]]+=1
            temp2_table.append(count_ls)
        return temp2_table

    def score(self,k,doc):
        copy_table=self.fit(self.X_train,self.y_train).copy()
        word_ls_copy=self.word_ls.copy()

        for word in doc.split():
            if word not in word_ls_copy:
                word_ls_copy.append(word)
                copy_table.append(np.zeros(len(set(self.y_train))))

        copy_table=np.array(copy_table)

        log_prior=np.log(np.array(copy_table).sum(axis=0)/np.array(copy_table).sum())
        log_likelyhood=np.log((copy_table+k)/(np.array(copy_table).sum(axis=0)+2*k))
        probs=np.exp(log_likelyhood[[i for i,v in enumerate(word_ls_copy) if v in doc.split()]].sum(axis=0)+log_prior)
        
        return probs/probs.sum()

    def predict(self,k,X_test):
        return np.array([self.score(k,doc).argmax() for doc in X_test])

In [3]:
data=pd.DataFrame({"tokens":["me free lottery","free get free you","you free scholarship","free to contact me","you won award","you ticket lottery"],
    "spam":[1,1,0,0,0,1]})
data

,tokens,spam
0,me free lottery,1
1,free get free you,1
2,you free scholarship,0
3,free to contact me,0
4,you won award,0
5,you ticket lottery,1


In [4]:
X_train=data["tokens"]
y_train=data["spam"]

In [7]:
model=naive_bayes()
model.fit(X_train,y_train)
model.score(0.5,"free lottery")

array([0.125, 0.875])

In [8]:
model.score(0.5,"free dksldkjf")

array([0.41666667, 0.58333333])

In [9]:
model.score(0.5,"ghi we wefwe")

array([0.5, 0.5])

In [10]:
from sklearn.datasets import fetch_20newsgroups
twenty_train=fetch_20newsgroups(subset="train",shuffle=True)
twenty_test=fetch_20newsgroups(subset="test",shuffle=True)

In [11]:
X_train,y_train=twenty_train.data[:100],twenty_train.target[:100]
X_test,y_test=twenty_test.data[:10],twenty_test.target[:10]

In [12]:
model=naive_bayes()
model.fit(X_train,y_train)
model.score(0.5,X_test[0])

array([4.08645878e-36, 4.67311253e-32, 1.84596700e-30, 5.32540548e-17,
       1.29803492e-20, 8.64589598e-13, 3.81217304e-18, 3.27257843e-42,
       3.05942430e-20, 2.05501656e-14, 4.38908423e-09, 3.00200712e-18,
       3.21204399e-24, 9.99999993e-01, 1.66407216e-51, 8.37337598e-10,
       2.55110419e-32, 1.69468763e-61, 1.92226990e-09, 1.17552462e-10])

In [14]:
prediction=model.predict(0.5,X_test)
prediction

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in true_divide


array([13,  0, 13,  0, 13,  0, 13, 13, 13, 13])

In [15]:
(prediction==y_test).mean() #결과 처참.........

0.0

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

text_clf=Pipeline([("vect",CountVectorizer()),
                   ("tfidf",TfidfTransformer()),
                   ("clf",MultinomialNB()),])
text_clf=text_clf.fit(twenty_train.data[:100],twenty_train.target[:100])
twenty_test=fetch_20newsgroups(subset="test",shuffle=True)
predicted=text_clf.predict(twenty_test.data[:10])
np.mean(predicted==twenty_test.target[:10])

0.1

다행히 sklearn도 결과가 그리 좋지 않아보임.. 텍스트가 복잡한것에 비해 데이터가 적어서 그런듯...